# Staffing Post Office

The post office needs a minimum number of employees for each day of the week:


| Day | M | T | W | R | F | S | S |
| ----| --- | ---|---|---|---|---|---|
| Index| 1|2|3|4|5|6|7|
| Min Required| 17 | 13 | 15 | 19 | 14 | 16 | 11|


Full-time office employees must work $5$ consecutive days, but can start their
``rotation'' on any day of the week

What is the minimum number of full-time employees required?



In [4]:
using NamedArrays

day_cover = [:Monday, :Tuesday, :Wednesday, :Thursday, :Friday, :Saturday, :Sunday]

start_shift = [:Monday, :Tuesday, :Wednesday, :Thursday, :Friday, :Saturday, :Sunday]

required = Dict(zip(day_cover, [17,13,15,19,14,16,11]))

shift_cover = [
    1 0 0 1 1 1 1
    1 1 0 0 1 1 1
    1 1 1 0 0 1 1 
    1 1 1 1 0 0 1
    1 1 1 1 1 0 0 
    0 1 1 1 1 1 0
    0 0 1 1 1 1 1
]

A_NA = NamedArray(shift_cover, (day_cover, start_shift), ("cover date", "Rotation start date")) ;

# Now can build the model to find the minimum number of employees

using JuMP, Clp
m = Model(Clp.Optimizer)
@variable(m, x[start_shift] >= 0)
@objective(m, Min, sum(x[j] for j in start_shift))
@constraint(m, [i in day_cover], sum(A_NA[i,j]*x[j] for j in start_shift) >= required[i])
optimize!(m)

using Formatting
printfmtln("Minimum number of employees {:1f}: ", objective_value(m))
for j in start_shift
    printfmtln("{1:1f} workers start on {2}", value(x[j]), j)
end

Minimum number of employees 22.333333: 
6.333333 workers start on :Monday
5.000000 workers start on :Tuesday
0.333333 workers start on :Wednesday
7.333333 workers start on :Thursday
0.000000 workers start on :Friday
3.333333 workers start on :Saturday
0.000000 workers start on :Sunday
Coin0506I Presolve 7 (0) rows, 7 (0) columns and 35 (0) elements
Clp0006I 0  Obj 0 Primal inf 105 (7)
Clp0006I 5  Obj 22.333333
Clp0000I Optimal - objective value 22.333333
Clp0032I Optimal objective 22.33333333 - 5 iterations time 0.002


Variation 1

* Employees make $1000/week
* Employees who work one weekend day make an extra $200/week
* Employees who work both weekend days make an extra $500/week

Just change the objective function coefficients:

| Shift Start | Weekly Pay | 
| --- | --- | 
| Monday | 1000 |
| Tuesday | 1200 | 
| Wednesday | 1500 | 
| Thursday | 1500 | 
| Friday | 1500 | 
| Saturday | 1500 | 
| Sunday | 1200 | 


In [5]:
costs = Dict(zip(start_shift, [1000, 1200, 1500, 1500, 1500, 1500, 1200]))

using JuMP, Clp
m = Model(Clp.Optimizer)
@variable(m, x[start_shift] >= 0)
@objective(m, Min, sum(costs[j]*x[j] for j in start_shift))
@constraint(m, [i in day_cover], sum(A_NA[i,j]*x[j] for j in start_shift) >= required[i])
optimize!(m)

using Formatting
printfmtln("Minimum Employee Cost {:1f}: ", objective_value(m))
for j in start_shift
    printfmtln("{1:1f} workers start on {2}", value(x[j]), j)
end

Minimum Employee Cost 28800.000000: 
6.000000 workers start on :Monday
5.333333 workers start on :Tuesday
0.000000 workers start on :Wednesday
7.333333 workers start on :Thursday
0.000000 workers start on :Friday
3.333333 workers start on :Saturday
0.333333 workers start on :Sunday
Coin0506I Presolve 7 (0) rows, 7 (0) columns and 35 (0) elements
Clp0006I 0  Obj 0 Primal inf 105 (7)
Clp0006I 5  Obj 28800
Clp0000I Optimal - objective value 28800
Clp0032I Optimal objective 28800 - 5 iterations time 0.002


Variation #2

* Part-time employees: Work two days/week, but most be consecutive.  Paid $200/day for weekday, $300/day for weekend
* No more than 10% of employees can be part-time

| Part Time Start | Pay |
| --- | --- | 
| Monday | 400 | 
| Tuesday | 400 | 
| Wednesday | 400 | 
| Thursday | 400 | 
| Friday | 500 | 
| Saturday | 600 | 
| Sunday | 500 |


In [6]:
# Here is a dictionary of the weekly (2 day) costs for part time employees
part_time_costs = Dict(zip(start_shift, [400, 400, 400, 400, 500, 600, 500]))

# Coverage matrix for part time employees
part_time_shift_cover = [
    1 0 0 0 0 0 1
    1 1 0 0 0 0 0
    0 1 1 0 0 0 0 
    0 0 1 1 0 0 0
    0 0 0 1 1 0 0 
    0 0 0 0 1 1 0
    0 0 0 0 0 1 1
]

B_NA = NamedArray(part_time_shift_cover, (day_cover, start_shift), ("cover date", "Part Time Rotation start date")) ;

using JuMP, Clp
m = Model(Clp.Optimizer)
@variable(m, x[start_shift] >= 0) # Regular workers start
@variable(m, y[start_shift] >= 0) # Part time workers start
@variable(m, u[day_cover] >= 0) # Regular workers working on day
@variable(m, v[day_cover] >= 0) # Part time workers working on day

@objective(m, Min, sum(costs[j]*x[j] + part_time_costs[j]*y[j] for j in start_shift))

@constraint(m, [i in day_cover], sum(A_NA[i,j]*x[j] for j in start_shift) == u[i])
@constraint(m, [i in day_cover], sum(B_NA[i,j]*y[j] for j in start_shift) == v[i])
@constraint(m, [i in day_cover], u[i] + v[i] >= required[i])

# You can try it with and without this constraint to see the difference imposed by the Union retriction
@constraint(m, [i in day_cover], v[i] <= 0.1 * u[i])

optimize!(m)

using Formatting
printfmtln("Minimum Employee Cost {:1f}: ", objective_value(m))
for j in start_shift
    if value(x[j]) > 0.01
        printfmtln("{1:1f} Regular Workers start on {2}", value(x[j]), j)
    end
    if value(y[j]) > 0.01
        printfmtln("{1:1f} Part Time Workers start on {2}", value(y[j]), j)
    end
end

printfmtln("Minimum Employee Cost {:1f}: ", objective_value(m))
for i in day_cover
    if value(u[i]) > 0.01
        printfmtln("{1:1f} Regular Working on {2}", value(u[i]), i)
    end
    if value(v[i]) > 0.01
        printfmtln("{1:1f} Part Time Working on {2}", value(v[i]), i)
    end
end



Minimum Employee Cost 28281.818182: 
Coin0506I Presolve 14 (-14) rows, 14 (-14) columns and 98 (7) elements
Clp0006I 0  Obj 0 Primal inf 104.99999 (7)
Clp0006I 10  Obj 28281.818
Clp0000I Optimal - objective value 28281.818
Coin0511I After Postsolve, objective 28281.818, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 28281.81818 - 10 iterations time 0.002, Presolve 0.00


LoadError: syntax: line break in ":" expression